In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('preprocessed.csv')

In [5]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")

True
NVIDIA GeForce RTX 2050


In [4]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124
  Using cached https://download.pytorch.org/whl/sympy-1.13.1-py3-none-any.whl (6.2 MB)
     ---------------------------------------- 0.0/536.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/536.2 kB ? eta -:--:--
     ---------------------------------------- 536.2/536.2 kB 5.8 MB/s  0:00:00
   ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 GB 2.8 MB/s eta 0:15:08
   ---------------------------------------- 0.0/2.5 GB 2.8 MB/s eta 0:15:08
   ---------------------------------------- 0.0/2.5 GB 3.0 MB/s eta 0:13:51
   ---------------------------------------- 0.0/2.5 GB 2.4 MB/s eta 0:17:37
   ---------------------------------------- 0.0/2.5 GB 2.2 MB/s eta 0:19:24
   ---------------------------------------- 0.0/2.5 GB 2.1 MB/s eta 0:20:24
   --------------------------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, classification_report

# ===============================
# 🔧 Device Setup (GPU / CPU)
# ===============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ===============================
# 📊 Example Dataset (replace with your df)
# ===============================
df = pd.read_csv("preprocessed.csv")  # your actual dataset path
target_col = "CVDCRHD4"  # heart attack column (1=Yes, 2=No)

# Drop missing / unknown labels
df = df[df[target_col].isin([1, 2])]
df[target_col] = df[target_col].replace({1: 1, 2: 0})

# Split features and target
X = df.drop(columns=[target_col])
y = df[target_col]

# Identify numeric and categorical columns
numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns
categorical_cols = X.select_dtypes(exclude=["int64", "float64"]).columns

from imblearn.over_sampling import SMOTE

# ===============================
# 🧹 Preprocessing
# ===============================
numeric_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='median')),
    ('scale', StandardScaler())
])
categorical_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numeric_cols),
    ('cat', categorical_pipeline, categorical_cols)
])

# Apply preprocessing
X_processed = preprocessor.fit_transform(X)

# Split before SMOTE (so test set is untouched)
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y, test_size=0.2, random_state=42, stratify=y
)

# ===============================
# ⚙️ Apply SMOTE to training data
# ===============================
smote = SMOTE(random_state=42, sampling_strategy='minority')
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

print("Before SMOTE:", np.bincount(y_train))
print("After SMOTE:", np.bincount(y_train_res))

# Convert to tensors
X_train_tensor = torch.tensor(X_train_res, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train_res.values, dtype=torch.long).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long).to(device)


# Create DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# ===============================
# 🧠 Model Definition
# ===============================
class AttentionLayer(nn.Module):
    def __init__(self, input_dim):
        super(AttentionLayer, self).__init__()
        self.attn = nn.Sequential(
            nn.Linear(input_dim, input_dim),
            nn.Tanh(),
            nn.Linear(input_dim, 1)
        )

    def forward(self, x):
        attn_weights = torch.softmax(self.attn(x), dim=1)
        context = torch.sum(attn_weights * x, dim=1)
        return context

class MLPWithAttention(nn.Module):
    def __init__(self, input_dim):
        super(MLPWithAttention, self).__init__()
        self.attn = AttentionLayer(input_dim)
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )

    def forward(self, x):
        context = self.attn(x)
        out = self.net(x)
        return out

model = MLPWithAttention(X_train.shape[1]).to(device)

# ===============================
# ⚙️ Training Setup
# ===============================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
epochs = 40

# ===============================
# 🚀 Training Loop
# ===============================
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")

# ===============================
# 🧾 Evaluation
# ===============================
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        preds = model(xb)
        y_true.extend(yb.cpu().numpy())
        y_pred.extend(torch.argmax(preds, dim=1).cpu().numpy())

acc = accuracy_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
print(f"\n✅ Accuracy: {acc*100:.2f}%")
print(f"📈 Recall: {recall*100:.2f}%")
print("\n📋 Classification Report:\n", classification_report(y_true, y_pred))


Using device: cuda


C:\Users\abhia\AppData\Local\Temp\ipykernel_39608\2299538647.py:71: DeprecationWarning: Non-integer input passed to bincount. In a future version of NumPy, this will be an error. (Deprecated NumPy 2.1)
  print("Before SMOTE:", np.bincount(y_train))
C:\Users\abhia\AppData\Local\Temp\ipykernel_39608\2299538647.py:72: DeprecationWarning: Non-integer input passed to bincount. In a future version of NumPy, this will be an error. (Deprecated NumPy 2.1)
  print("After SMOTE:", np.bincount(y_train_res))


Before SMOTE: [145302   7561]
After SMOTE: [145302 145302]
Epoch 1/40, Loss: 0.4550
Epoch 2/40, Loss: 0.4349
Epoch 3/40, Loss: 0.4282
Epoch 4/40, Loss: 0.4236
Epoch 5/40, Loss: 0.4193
Epoch 6/40, Loss: 0.4155
Epoch 7/40, Loss: 0.4116
Epoch 8/40, Loss: 0.4089
Epoch 9/40, Loss: 0.4057
Epoch 10/40, Loss: 0.4033
Epoch 11/40, Loss: 0.3999
Epoch 12/40, Loss: 0.3973
Epoch 13/40, Loss: 0.3938
Epoch 14/40, Loss: 0.3917
Epoch 15/40, Loss: 0.3892
Epoch 16/40, Loss: 0.3873
Epoch 17/40, Loss: 0.3845
Epoch 18/40, Loss: 0.3823
Epoch 19/40, Loss: 0.3803
Epoch 20/40, Loss: 0.3782
Epoch 21/40, Loss: 0.3760
Epoch 22/40, Loss: 0.3747
Epoch 23/40, Loss: 0.3728
Epoch 24/40, Loss: 0.3709
Epoch 25/40, Loss: 0.3688
Epoch 26/40, Loss: 0.3670
Epoch 27/40, Loss: 0.3659
Epoch 28/40, Loss: 0.3642
Epoch 29/40, Loss: 0.3632
Epoch 30/40, Loss: 0.3618
Epoch 31/40, Loss: 0.3603
Epoch 32/40, Loss: 0.3590
Epoch 33/40, Loss: 0.3579
Epoch 34/40, Loss: 0.3562
Epoch 35/40, Loss: 0.3551
Epoch 36/40, Loss: 0.3543
Epoch 37/40, L